`yield from`
===

In [ ]:
def braced_chain(*gens, before='{', after='}'):
    for gen in gens:
        yield before
        for x in gen:
            yield x
        yield after

In [ ]:
g2 = lambda: (x**2 for x in range(3))
g3 = lambda: (x**3 for x in range(3))
g4 = lambda: (x**4 for x in range(3))

In [ ]:
list(braced_chain(g2(), g3(), g4(), before='<', after='>'))

In [ ]:
def braced_chain(*gens, before='{', after='}'):
    for gen in gens:
        yield before
        yield from gen
        yield after

Корутины
===

In [ ]:
class Mean:
    def __init__(self):
        self._sum = 0
        self._n = 0
        
    def add(self, x):
        self._sum += x
        self._n += 1
        
        return x >= self._sum / self._n

In [ ]:
m = Mean()

In [ ]:
m.add(1)

In [ ]:
m.add(10)

In [ ]:
m.add(2)

In [ ]:
m.add(3)

In [ ]:
m.add(4)

In [ ]:
def mean():
    result = None
    s = 0
    n = 0
    while True:
        x = yield result
        s += x
        n += 1
        result = x >= s / n

In [ ]:
m = mean()

In [ ]:
m

In [ ]:
next(m)

In [ ]:
m.send(1)

In [ ]:
m.send(10)

In [ ]:
m.send(2)

In [ ]:
m.send(3)

In [ ]:
m.send(4)

In [ ]:
def mean():
    result = None
    s = 0
    n = 0
    while True:
        x = yield result
        if x is None:
            break
        s += x
        n += 1
        result = x >= s / n

In [ ]:
m = mean()

In [ ]:
next(m)

In [ ]:
m.send(1)

In [ ]:
m.send(2)

In [ ]:
m.send(None)

`RestoringGet`
===

In [ ]:
logger = logging.getLogger(name)


class RestoringGet:
    MAX_RESTORES = 10

    def init(self, url, get_kwargs=None):
        if get_kwargs is None:
            get_kwargs = {}

        self._url = url
        self._kwargs = get_kwargs

    def get_generator(self):
        restores = 0
        offset = 0
        headers = {}
        while True:
            restores += 1
            if restores > self.MAX_RESTORES:
                raise TooManyRestores()

            response = requests.get(
                self._url, headers=headers, **self._kwargs)
            response.raise_for_status()

            real_length = yield response

            content_length = parse_int(
                response.headers.get('Content-Length'), None)
            if content_length is None \
                or content_length + offset <= real_length:
                break

            logger.info(
                'GET looks to be interrupted, trying to continue')
            offset = real_length
            headers = {'Range': 'bytes={}-'.format(offset)}

In [ ]:
gen = RestoringGet(...).get_generator()
response = next(gen)
while response:
    file = save_response(response)
    try:
        response = gen.send(file.size)
    except StopIteration:
        response = None

`yield from` для корутин
===

In [ ]:
def braced_chain(*gens, before='{', after='}'):
    for gen in gens:
        yield before
        for x in gen:
            yield x
        yield after

In [ ]:
b = braced_chain(g2(), mean())

In [ ]:
list(b)

In [ ]:
def braced_chain(*gens, before='{', after='}'):
    for gen in gens:
        yield before
        yield from gen
        yield after

In [ ]:
m = mean()
b = braced_chain(g2(), m)
(
    next(b), next(b), next(b), next(b), next(b),
    next(b), next(b), b.send(1), b.send(2), b.send(None)
)

In [ ]:
b.send(13)

`return` в генераторе
===

In [ ]:
def mean():
    result = None
    s = 0
    n = 0
    while True:
        x = yield result
        if x is None:
            break
        s += x
        n += 1
        result = x >= s / n
    return s / n

In [ ]:
m = mean()
next(m), m.send(1), m.send(2), m.send(None)

In [ ]:
def braced_chain(*gens, before='{', after='}'):
    results = []
    for gen in gens:
        yield before
        result = yield from gen
        results.append(result)
        yield after
    return results

In [ ]:
m = mean()
b = braced_chain(g2(), m)
(
    next(b), next(b), next(b), next(b), next(b),
    next(b), next(b), b.send(1), b.send(2), b.send(None)
)

In [ ]:
next(b)

Communicate
===

In [ ]:
import subprocess

In [ ]:
p = subprocess.run(['python', '-c', 'open("/tmp/5555", "w").close()'])

In [ ]:
p = subprocess.run(
    ['perl', '-E', 'say "out"; warn "err\n"'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

In [ ]:
p.stdout, p.stderr

In [ ]:
from subprocess import Popen

In [ ]:
p = Popen(
    ['perl', '-E', 'say "out"; warn "err\n"'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

In [ ]:
p.communicate()

In [ ]:
while True:
    out = p.stdout.readline()
    err = p.stderr.readline()
    if out:
        print(f'out: {out}')
    if err:
        print(f'err: {out}')
    if not out and not err:
        break       

In [ ]:
p = Popen(
    ['perl', '-E', 'warn "err\n" x 1000000; say "out";'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

Threads
===

In [ ]:
import threading
threads_num = 4

p = Popen(
    ['perl', '-E', 'warn "err\n"; say "out";'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

def log(fh, prefix):
    while True:
        line = fh.readline()
        if line:
            print('{}: {}'.format(prefix, line.decode('utf8')), end='')
        else:
            break

threads = [
    threading.Thread(target=log, args=(p.stdout, 'out')),
    threading.Thread(target=log, args=(p.stderr, 'err')),
]
for t in threads:
    t.start()
for t in threads:
    t.join()

Selector
===

In [ ]:
import selectors

sel = selectors.DefaultSelector()

In [ ]:
sel

In [ ]:
p = Popen(
    ['perl', '-E', 'warn "err\n"; say "out"; warn "2\n"'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)
sel.register(p.stdout, selectors.EVENT_READ, lambda l: print('out: {}'.format(l)))
sel.register(p.stderr, selectors.EVENT_READ, lambda l: print('err: {}'.format(l)))

In [ ]:
while True:
    events = sel.select()
    if not events:
        break
    for key, mask in events:
        callback = key.data
        line = key.fileobj.readline()
        if line:
            callback(line)
        else:
            sel.unregister(key.fileobj)

`asyncio`
===

In [ ]:
import asyncio
loop = asyncio.get_event_loop()

In [ ]:
p = asyncio.create_subprocess_exec(
    'perl', '-E', 'warn "err\n"; say "out"; warn "2\n"',
    stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
)

In [ ]:
p

In [ ]:
next(p)

In [ ]:
@asyncio.coroutine
def execute():
    p = yield from asyncio.create_subprocess_exec(
        'perl', '-E', 'warn "err\n"; say "out"; warn "2\n"',
        stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
    )
    print('>>> {}'.format(p))

In [ ]:
execute()

In [ ]:
loop.run_until_complete(execute())

In [ ]:
@asyncio.coroutine
def log(fh, prefix):
    while True:
        line = yield from fh.readline()  ###
        if line:
            print('{}: {}'.format(prefix, line.decode('utf8')), end='')
        else:
            break

In [ ]:
@asyncio.coroutine
def execute(loop):
    p = yield from asyncio.create_subprocess_exec(
        'perl', '-E', 'warn "err\n"; say "out"; warn "2\n"',
        stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
    )
    
    loop.create_task(log(p.stdout, 'stdout'))
    loop.create_task(log(p.stderr, 'stderr'))
    
    yield from p.wait()

In [ ]:
loop.run_until_complete(execute(loop))

`async` / `await`
===

In [ ]:
async def log(fh, prefix):
    while True:
        line = await fh.readline()
        if line:
            print('{}: {}'.format(prefix, line))
        else:
            break

In [ ]:
async def execute(loop):
    p = await asyncio.create_subprocess_exec(
        'perl', '-E', 'warn "err\n"; say "out"; warn "2\n"',
        stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
    )
    
    loop.create_task(log(p.stdout, 'stdout'))
    loop.create_task(log(p.stderr, 'stderr'))
    
    await p.wait()

In [ ]:
loop.run_until_complete(execute(loop))

`aiohttp`
===

In [ ]:
import aiohttp
import asyncio

async def fetch(session, url):
    async with session.get(url) as response:
        if response.status == 200:
            return await response.text()  # return yield from
        else:
            return f'ERROR: {response.status}'

async def download_wiki(article):
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, 'http://de.wikipedia.org/{}'.format(article))
        return html[:15]

loop = asyncio.get_event_loop()
tasks = asyncio.gather(
    download_wiki('wiki/Évariste_Galois'),
    download_wiki('wiki/Alan_Turing'),
    download_wiki('zzz'),
)

In [ ]:
tasks

In [ ]:
loop.run_until_complete(tasks)

Сервер
===

In [ ]:
from aiohttp import web